In [ ]:
"""This code calculated to use a high res. flood depth tiffs
to get wave heights modified 
Written by Taran Kalra, August 19, 2024"""


In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat              
import rioxarray
import rasterio 
#from create_raster import make_swan_raster_StructuredGrid

In [2]:
cases = ["1perc", "2perc", "5perc", "10perc", "20perc", "50perc"]
years = ["100yr", "50yr", "20year", "10year", "5year", "2year"] # year of return 
max_wave_height = [5, 5, 5 , 5, 5, 5]

In [149]:
a= -0.636
b = 8.675

a = -0.572
b= 1.2416

In [150]:
xx = [1e-6, 1e-5, 3.0000e-05, 1e-4, 1e-3, 1e-2, 1e-1,1]

#yy = np.array(xx)
#xx = np.array(xx)

In [151]:
yy=[]

In [152]:
320/6


53.333333333333336

In [153]:
for i in range(len(xx)):
    #print(xx)
    print(xx[i])
    y = a*np.log(xx[i])+b
    yy.append(y)

1e-06
1e-05
3e-05
0.0001
0.001
0.01
0.1
1


In [154]:
yy=np.round(yy,2)
print(yy)
print(xx)

[9.14 7.83 7.2  6.51 5.19 3.88 2.56 1.24]
[1e-06, 1e-05, 3e-05, 0.0001, 0.001, 0.01, 0.1, 1]


In [99]:
1/(365*24*19)

6.00817111271329e-06

In [128]:
xx = [1e-3, 1e-4,  1e-5]
yy=[]

In [129]:
for i in range(len(xx)):
    #print(xx)
    #prinxx[i])
    y = np.log(xx[i])
    yy.append(y)
    print(y)

-6.907755278982137
-9.210340371976182
-11.512925464970229


In [57]:
# Define the logarithmic function
def log_func(x, a, b):
    return a * np.log(x) + b

In [66]:
from scipy.optimize import curve_fit
 

In [70]:
from scipy.optimize import least_squares


In [130]:

# Define the logarithmic function
def log_func(params, x, y):
    a, b = params
    return a * np.log(x) + b - np.log(y)

# Sample data
x = np.array([1, 10, 100])
y = np.array([1e-3, 1e-4, 1e-5])

# Transform the data
x_log = np.log(x)
y_log = np.log(y)

# Initial guess for parameters
params0 = [1, 1]

# Define the residual function
def residual(params):
    return log_func(params, x, y)

# Fit the curve using least_squares
result = least_squares(residual, params0)

print("Fitted parameters: a = {:.2f}, b = {:.2f}".format(*result.x))

Fitted parameters: a = -1.00, b = -6.91


In [132]:

import numpy as np
from scipy.optimize import least_squares

# ... (rest of the code remains the same)

# Fitted parameters
a, b = result.x

# Predict output for x = 10
x_pred = 20
y_pred = a * np.log(x_pred) + b

print("Predicted output for x = 10: {:.4e}".format(np.exp(y_pred)))

Predicted output for x = 10: 5.0000e-05


In [ ]:


# Plot the data and the fitted curve
import matplotlib.pyplot as plt

plt.plot(x, y, 'bo', label='Data')
plt.plot(x_fit, y_fit, 'r-', label='Fitted curve')
plt.legend()
plt.show()

In [ ]:
breaking_wave_coefficient = 0.78 

In [ ]:
# Loop over the filenames in the directory
for i, case in enumerate(cases): 
    outfile = f'{output_directory}\{case}_{scenario_out}_wave_height.tif' 
 
    print(gridfile) 
        
    # Grid file path specified 
    # this is the finer TIFF file 
    finer_xds = rioxarray.open_rasterio(gridfile) 
    depth = finer_xds.squeeze() 

    # This is different for each pixel 
    breaking_wave_depth_criteria = 1.3*depth 

    # Send the wave height for each scenario 
    #Hs_mod = modify_wave_height(max_wave_height, depth, breaking_wave_coefficient , breaking_wave_depth_criteria)

    breaking_wave_height = breaking_wave_coefficient * (depth)

    ds_new = depth*0.0
    modified_max_wave_height = max_wave_height[i] + ds_new # to convert a raster array of max wave height over the domain
    print(type(modified_max_wave_height))
    modified_max_wave_height = np.where(depth<=breaking_wave_depth_criteria, breaking_wave_height, max_wave_height[i])

    # Fill the array to get modified wave height 
    ds_new = modified_max_wave_height + ds_new # Add 0's through ds_new to make sure it is a xarray object that can be converted through rio command 
    print(type(ds_new))
    print(outfile)
  
    ds_new.rio.to_raster(outfile) # save the new array in a raster before changing the profile of the raster

    print(np.nanmin(ds_new))
    print(np.nanmax(ds_new)) 

In [ ]:

#fig = plt.figure(figsize=(15,10))
#ax = fig.add_subplot(1,1,1)
#z1_plot = ax.pcolormesh(fine_data.x.values, fine_data.y.values, array, cmap='viridis', vmin=-20, vmax=20)
#plt.colorbar(z1_plot)